In [ ]:
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa
from IPython.display import Audio
%matplotlib inline

import audio_utils
import vamp

import os

os.environ["VAMP_PATH"] = "./melodia_plugins/"

In [ ]:
def melodic_frequency_filepath(filepath):
    audio, sr = librosa.load(filepath, mono=True)
    data = vamp.collect(audio, sr, "mtg-melodia:melodia")
    hop, melody = data['vector']
    return melody

def melodic_frequency(audio, sr):
    data = vamp.collect(audio, sr, "mtg-melodia:melodia")
    hop, melody = data['vector']
    return melody

In [ ]:
# A clearer option is to get rid of the negative values before plotting
def plot_melody(melody):
    timestamps = 8 * 128/44100.0 + np.arange(len(melody)) * (128/44100.0)
    melody_pos = melody[:]
    melody_pos[melody<=0] = None
    plt.figure(figsize=(18,6))
    plt.plot(timestamps, melody_pos)
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.show()

In [ ]:
HOP = 128.0

def sample_to_index(sample):
    index = sample/HOP - 8
    
    return int(index)

def find_interval_freq(start_sample, end_sample, melody):
    start_index = sample_to_index(start_sample)
    end_index = sample_to_index(end_sample)
    
    relevant_frequencies = melody[start_index:end_index]
    
    freq = find_freq(relevant_frequencies)
    
    return freq

def find_freq(melody):
    voiced_frequencies = melody[melody > 0]
    freq = np.median(voiced_frequencies)
    return freq

In [ ]:
def plot_and_play(y, sr, label=''):
    print(label)
    IPython.display.display(Audio(y, rate=sr))
    audio_utils.plot_audio(y, sr=sr)

In [ ]:
from pydub import AudioSegment
import subprocess

SPEECH_FILEPATH = 'temp/tts_dump.wav'
TEXT_FILEPATH = 'temp/text.txt'
SCRIPT_FILEPATH = "./text_gen.sh"

def tts(text):
    """
        returns list of AudioSegments corresponding to each word in text
    """
    tokens = text.split(' ')
    
    audio_chunks = [tts_word(token) for token in tokens]

    return audio_chunks

def tts_word(text, sr):

    with open(TEXT_FILEPATH, 'w+') as f:
        f.write(text)

    p = subprocess.call(SCRIPT_FILEPATH, shell=True)

    y = AudioSegment.from_wav(SPEECH_FILEPATH)
    
    y = y.set_frame_rate(sr)

    return y

In [ ]:
from pydub.silence import split_on_silence

def get_syllable_signals(text, sr):
    params = {'min_silence_len': 40, 
              'silence_thresh': -18, 
              'keep_silence': 100
             }
    
    audio_words = tts_word(text, sr)

    audio_chunks = split_on_silence(audio_words, 
                                    min_silence_len=params['min_silence_len'], 
                                    silence_thresh=params['silence_thresh'], 
                                    keep_silence=params['keep_silence'])

    all_segments = [np.array(chunk.get_array_of_samples(), dtype=float) for chunk in audio_chunks]

    return all_segments

In [ ]:
def louisa():
    print(onset_times.shape)
    print("onsets with clicks")
    clicks = librosa.clicks(times=onset_times, sr=sr, length=len(y))
    plot_and_play(clicks + y, sr)
    plt.figure()
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0, onset_env.max(), color='r', alpha=0.8, label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    print(peaks)
    print(onset_times.shape)

In [ ]:
import madmom

def onset_intervals(y, sr):
    proc = madmom.features.onsets.OnsetPeakPickingProcessor()
    act = madmom.features.onsets.RNNOnsetProcessor()(y)
    onset_times = proc(act)

    onset_samples = librosa.core.time_to_samples(onset_times, sr=sr)
    starts = onset_samples[:len(onset_samples)-1]
    ends = np.append(onset_samples[1:], [len(y)])
    intervals = zip(starts, ends)
    return intervals
    
def onset_frequencies(y, song, sr):
    intervals = onset_intervals(y, sr)
    y_melody = melodic_frequency(y, sr)
    y_freqs = [find_interval_freq(start, end, y_melody) for start, end in intervals]
    print(y_freqs)
    intervals, y_freqs = zip(*[(i, f) for i, f in zip(intervals, y_freqs) if not np.isnan(f)])

    return intervals, y_freqs

In [ ]:
def pitch_difference(fn, f0):
    n_steps = np.round(np.log2(float(fn)/f0) * 12)
    return n_steps

In [ ]:
def read_voice_frequency():
    filepath = 'espeak-data/voices/robot'
    with open(filepath) as f:
        lines = [line.split(' ') for line in f.readlines()]
        line = [line for line in lines if line and line[0] == 'pitch'][0]
        f0 = int(line[1])
        return f0

In [ ]:
def autotune(signals, fn, sr):
    
    plot_and_play(np.concatenate(signals), sr)
    shifted_signals = [pitch_correct(y=y, fn=f, sr=sr) for y, f in zip(signals, fn)]
    
    return shifted_signals

def pitch_correct(y, fn, sr):
    f0 = read_voice_frequency()
    
    n_steps = pitch_difference(fn=fn, f0=f0)

    shifted_y = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=n_steps)

    return shifted_y    

In [ ]:
def stretch_signals(signals, intervals):
    stretchy = [stretch_signal(y=y, interval=interval) for y, interval in zip(signals, intervals)]
    return stretchy

def stretch_signal(y, interval):
    note_duration = interval[1] - interval[0]
    
    rate = len(y) / float(note_duration)
    stretched_y = librosa.effects.time_stretch(y, rate=rate)
    
    return stretched_y

In [ ]:
import nussl

def get_melody(y, sr):
    signal = nussl.AudioSignal(audio_data_array=y, sample_rate=sr)
    
    melodia = nussl.separation.melodia.Melodia(signal)

    melodia.run()

    foreground_and_background = melodia.make_audio_signals()
    
    foreground = foreground_and_background[1]
    
    background = foreground_and_background[0]
    
    melody = foreground.to_mono()
    
    backtrack = background.to_mono()
    
    return melody, backtrack

In [ ]:
def normalize_length(lst1, lst2, lst3):
    
    length = min(len(lst1), len(lst2), len(lst3))
    
    return lst1[:length], lst2[:length], lst3[:length]

In [ ]:
def overlay_signals(song_signal, vocal_signal):
    
    min_length = min(len(song_signal), len(vocal_signal))
    
    song_normal = librosa.util.normalize(song_signal)
    
    vocals_normal = librosa.util.normalize(vocal_signal)
    
    both = song_normal[:min_length] + vocals_normal[:min_length]
    
    return both

In [ ]:
def construct_voice(intervals, signals, y):
    padded_signals = [pad_signal(i, s, y) for i, s in zip(intervals, signals)]
    voice_signal = np.sum(padded_signals, axis=0)
    return voice_signal
        
def pad_signal(interval, signal, y):
    start, end = interval
    start_padding = np.zeros(start)
    end_padding = np.zeros(len(y)-len(signal)-start)
    padded_signal = np.concatenate([start_padding, signal, end_padding])
    return padded_signal

In [ ]:
def e2e(filename, text, sr):
    song, sr = librosa.load(filename, sr=sr)
    
    melody, background = get_melody(song, sr)
    
    intervals, frequencies = onset_frequencies(melody, song, sr)
    
    syllables = get_syllable_signals(text, sr)
    
    syllables, intervals, frequencies = normalize_length(syllables, intervals, frequencies)
    
    pitch_corrected_syllables = autotune(syllables, fn=frequencies, sr=sr)
    
    time_corrected_syllables = stretch_signals(pitch_corrected_syllables, intervals)
    
    tuned_voice_signal = construct_voice(intervals, time_corrected_syllables, y=song)
    
    song_with_voice = overlay_signals(background, tuned_voice_signal)
    
    return song_with_voice    

In [ ]:
sr = 44100
# f=open("twinkle_lyrics.txt", "r")
# if f.mode == 'r':
#     text =f.read()

    
# text = 'happy birthday to you'
y = e2e(filename='audio/twinkle.wav', text=text, sr=sr)

plot_and_play(y, sr)

# librosa.output.write_wav('redbone_output.wav', y, sr)